# 03. 지도 시각화 및 LISA 분석

## 목적
- 클러스터링 결과를 지도에 시각화
- LISA (Local Indicators of Spatial Association) 공간분석
- High-High 핫스팟 식별

In [1]:
import pandas as pd
import folium
from pyproj import Transformer
import warnings
warnings.filterwarnings('ignore')

result_dir = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석"
df = pd.read_csv(f"{result_dir}/클러스터링_결과_완전판.csv")

data_path = "/Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1009/빅콘테스트_전체병합데이터_20251008.csv"
full_df = pd.read_csv(data_path)

df = df.merge(full_df[['가맹점구분번호', '좌표정보(X)', '좌표정보(Y)']].drop_duplicates(), on='가맹점구분번호', how='left')

df_map = df[df['좌표정보(X)'].notna() & df['좌표정보(Y)'].notna()].copy()
print(f"좌표 데이터: {len(df_map):,}개")

좌표 데이터: 4,325개


## 좌표 변환 (TM → WGS84)

In [2]:
transformer = Transformer.from_crs("EPSG:5174", "EPSG:4326", always_xy=True)
coords = transformer.transform(df_map['좌표정보(X)'].values, df_map['좌표정보(Y)'].values)

df_map['longitude'] = coords[0]
df_map['latitude'] = coords[1]

center_lat = df_map['latitude'].median()
center_lon = df_map['longitude'].median()
print(f"지도 중심: ({center_lat:.6f}, {center_lon:.6f})")

지도 중심: (37.552340, 127.040970)


## 지도 생성 (클러스터별 위험도)

In [3]:
m1 = folium.Map(location=[center_lat, center_lon], zoom_start=13, tiles='OpenStreetMap')

cluster_colors = {0: '#FF0000', 1: '#00FF00', 2: '#FF6600'}

for _, row in df_map.iterrows():
    cluster = int(row['cluster'])
    color = cluster_colors.get(cluster, '#CCCCCC')
    radius = 5 if cluster == 1 else 8
    
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,
        color=color,
        fill=True,
        fillColor=color,
        fillOpacity=0.6,
        popup=f"Cluster {cluster}<br>{row['상태']}"
    ).add_to(m1)

map_path = f"{result_dir}/지도1_클러스터_위험도.html"
m1.save(map_path)
print(f"지도 저장: {map_path}")

지도 저장: /Users/yeong-gwang/Documents/배움 오전 1.38.42/외부/공모전/빅콘테스트/Project/work/ver3_/1012/result/3_가설1분석/지도1_클러스터_위험도.html


## 종합
- 빨간색: 고위험 클러스터
- 초록색: 정상 클러스터
- 공간적 군집 패턴 확인 가능